In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

import time
import os
import pandas as pd
from datetime import datetime, timedelta
from selenium.webdriver.common.by import By

In [45]:
from selenium.webdriver.support.wait import WebDriverWait

In [2]:
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_mac64.zip
Driver has been saved in cache [/Users/sasipim/.wdm/drivers/chromedriver/mac64/99.0.4844.51]


In [76]:
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

In [10]:
PATH = './Saraburi/test/saraburi_test.csv'
SAVE_PATH = './Saraburi/test/saraburi_test_wind_temp.csv'
LAT = '14.685833'
LONG = '100.871996'

In [9]:
data = pd.read_csv(PATH)
data = data.rename(columns={"Unnamed: 0": "date"})
data.head()

,date,PM2.5
0,2020-07-01 00:00:00,2.0
1,2020-07-01 01:00:00,NaN
2,2020-07-01 02:00:00,NaN
3,2020-07-01 03:00:00,3.0
4,2020-07-01 04:00:00,5.0


In [11]:
data['date_utc'] = pd.to_datetime(data['date']) - timedelta(hours=7)
data['url_date'] = data['date_utc'].apply(lambda x : x.strftime("%Y/%m/%d/%H%M")+"Z")

In [80]:
def getTemp(lat,long,date):
    url = "https://earth.nullschool.net/#"+date+'/wind/surface/level/overlay=temp/orthographic=97.73,-347.68,3906/loc='+ long+ ','+lat
    driver.get(url=url)
    time.sleep(2)
    html = driver.execute_script("return document.documentElement.outerHTML")
    # element = WebDriverWait(driver, 20).until(lambda x: x.find_elements(by=By.CSS_SELECTOR, value='#spotlight-panel.panel.card'))
    all_link = driver.find_elements(by=By.CSS_SELECTOR, value='#spotlight-panel.panel.card')
    for link in all_link:
        return link.text.split("\n")[-2]

In [81]:
def getWind(lat,long,date):
    url = "https://earth.nullschool.net/#"+date+'/wind/isobaric/850hPa/orthographic=97.73,-347.68,3906/loc='+ long+ ','+lat
    driver.get(url=url)
    time.sleep(2)
    html = driver.execute_script("return document.documentElement.outerHTML")
    # element = WebDriverWait(driver, 20).until(lambda x: x.find_elements(by=By.CSS_SELECTOR, value='#spotlight-panel.panel.card'))
    all_link = driver.find_elements(by=By.CSS_SELECTOR, value='#spotlight-panel.panel.card')
    for link in all_link:
        return link.text.split("\n")[-2]

In [68]:
x = data['url_date'][4]
p = getWind(LAT,LONG,x)
p

'255° @ 41'

In [82]:
data['temp'] = data['url_date'].progress_apply(lambda x : getTemp(LAT,LONG,x))

  0%|          | 0/8784 [00:00<?, ?it/s]

In [83]:
data['wind']=data['url_date'].progress_apply(lambda x : getWind(LAT,LONG,x))

  0%|          | 0/8784 [00:00<?, ?it/s]

In [84]:
data.to_csv(SAVE_PATH,index=False)

In [86]:
data1 = pd.read_csv(SAVE_PATH)
data1

,date,PM2.5,date_utc,url_date,temp,wind
0,2020-07-01 00:00:00,2.0,2020-06-30 17:00:00,2020/06/30/1700Z,25.9,250° @ 41
1,2020-07-01 01:00:00,NaN,2020-06-30 18:00:00,2020/06/30/1800Z,25.5,255° @ 41
2,2020-07-01 02:00:00,NaN,2020-06-30 19:00:00,2020/06/30/1900Z,25.5,255° @ 41
3,2020-07-01 03:00:00,3.0,2020-06-30 20:00:00,2020/06/30/2000Z,25.5,255° @ 41
4,2020-07-01 04:00:00,5.0,2020-06-30 21:00:00,2020/06/30/2100Z,25.1,265° @ 49
...,...,...,...,...,...,...
8779,2021-07-01 19:00:00,NaN,2021-07-01 12:00:00,2021/07/01/1200Z,29.7,245° @ 22
8780,2021-07-01 20:00:00,13.0,2021-07-01 13:00:00,2021/07/01/1300Z,29.7,235° @ 24
8781,2021-07-01 21:00:00,NaN,2021-07-01 14:00:00,2021/07/01/1400Z,29.7,240° @ 22
8782,2021-07-01 22:00:00,14.0,2021-07-01 15:00:00,2021/07/01/1500Z,29.7,240° @ 25
